In [1]:
import tflearn
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn import preprocessing
import chess_game as cg
import time

/Users/pschale/anaconda/envs/tf/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


Instructions for updating:
Use the retry module or similar alternatives.


In [18]:
board = tflearn.layers.core.input_data(shape=(None, 64, 8), name='board_input')
aux = tflearn.layers.core.input_data(shape=(None, 7), name='aux_input')
Y = tflearn.layers.core.input_data(shape=(None, 3), name='labels')

board_input_reshaped = tf.reshape(board, [-1, 8, 8, 8])

conv4 = tflearn.conv_2d(board_input_reshaped, 32, 4, activation='relu')


conv2 = tflearn.conv_2d(board_input_reshaped, 32, 2, activation='relu')
conv2_4 = tflearn.conv_2d(conv2, 64, 4, activation='relu')
# note: this is how to get separate branches of a larger NN and then have them converge
fc1 = tflearn.layers.core.fully_connected(tf.concat([conv4, conv2_4], 3), n_units=1000, activation='relu')

fc2 = tflearn.layers.core.fully_connected(tf.concat([fc1, aux], 1), n_units=2000, activation='relu')

net = tflearn.layers.core.fully_connected(fc2, n_units=3, activation='softmax')

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=net, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.00001).minimize(loss)

init = tf.global_variables_initializer()
#net = tflearn.regression(net, optimizer='adam', metric=loss,
#                         loss='categorical_crossentropy')
#model = tflearn.DNN(net, tensorboard_verbose=0)


In [3]:
a = pd.read_csv('board_positions/board_positions_1.csv')
a = a[~a.isnull().any(axis=1)]
a = a[a['moves_in_game'] - a['move_num']<10]
le = preprocessing.LabelEncoder()
le.fit(['k', 'q', 'r', 'b', 'n', 'p', ' ', 'P', 'N', 'B', 'R', 'Q', 'K'])

bcols = a.columns[:64]

a[bcols] = le.transform(a[bcols])
    

In [4]:
def get_training_data(data):
    
    d = data.sample(100)
    train = d[d.columns[:71]]
    board = train[train.columns[:64]]
    aux = train[train.columns[64:]]
    board_onehot = np.zeros((100, 64, 8))
    board_onehot[:, :, 0] = board.isin([7, 8, 9, 10, 11, 12])
    board_onehot[:, :, 1] = board.isin([0, 1, 2, 3, 4, 5])
    board_onehot[:, :, 2] = board.isin([0, 12])
    board_onehot[:, :, 3] = board.isin([1, 11])
    board_onehot[:, :, 4] = board.isin([2, 10])
    board_onehot[:, :, 5] = board.isin([3, 9])
    board_onehot[:, :, 6] = board.isin([4, 8])
    board_onehot[:, :, 7] = board.isin([5, 7])
    labels = np.zeros((100,3))
    labels[d['winner']==-1, 0] = 1
    labels[d['winner']==0, 1] = 1
    labels[d['winner']==1, 2] = 1
    
    return board_onehot, aux, labels
    

In [21]:
saver = tf.train.Saver()

sess = tf.Session()
sess.run(init)

for epoch in range(30):  # 2 epochs
    for i in range(200):
        batch_board, batch_aux, batch_ys = get_training_data(a)
        sess.run(optimizer, feed_dict={board: batch_board, aux: batch_aux, Y: batch_ys})
        cost = sess.run(loss, feed_dict={board: batch_board, aux: batch_aux, Y: batch_ys})

        if i % 20 == 0:
            print("Epoch:", '%03d' % (epoch + 1), "Step:", '%03d' % i,
                  "Loss:", str(cost))

Epoch: 001 Step: 000 Loss: 1.1061362
Epoch: 001 Step: 020 Loss: 1.0445443
Epoch: 001 Step: 040 Loss: 1.0402567
Epoch: 001 Step: 060 Loss: 1.0193391
Epoch: 001 Step: 080 Loss: 0.99248093
Epoch: 001 Step: 100 Loss: 0.96708727
Epoch: 001 Step: 120 Loss: 0.9682187
Epoch: 001 Step: 140 Loss: 0.9495423
Epoch: 001 Step: 160 Loss: 0.96677107
Epoch: 001 Step: 180 Loss: 0.88641965
Epoch: 002 Step: 000 Loss: 0.97250783
Epoch: 002 Step: 020 Loss: 0.91933334
Epoch: 002 Step: 040 Loss: 0.9124889
Epoch: 002 Step: 060 Loss: 0.9255668
Epoch: 002 Step: 080 Loss: 0.9032878
Epoch: 002 Step: 100 Loss: 0.89139205
Epoch: 002 Step: 120 Loss: 0.890964
Epoch: 002 Step: 140 Loss: 0.95472413
Epoch: 002 Step: 160 Loss: 0.95608824
Epoch: 002 Step: 180 Loss: 0.943003
Epoch: 003 Step: 000 Loss: 0.9093886
Epoch: 003 Step: 020 Loss: 0.9260199
Epoch: 003 Step: 040 Loss: 0.90726995
Epoch: 003 Step: 060 Loss: 0.8952315
Epoch: 003 Step: 080 Loss: 0.92401034
Epoch: 003 Step: 100 Loss: 0.8851005
Epoch: 003 Step: 120 Loss: 0.

Epoch: 023 Step: 000 Loss: 0.782693
Epoch: 023 Step: 020 Loss: 0.8113545
Epoch: 023 Step: 040 Loss: 0.77444476
Epoch: 023 Step: 060 Loss: 0.7981622
Epoch: 023 Step: 080 Loss: 0.7650945
Epoch: 023 Step: 100 Loss: 0.81216574
Epoch: 023 Step: 120 Loss: 0.79549235
Epoch: 023 Step: 140 Loss: 0.7883401
Epoch: 023 Step: 160 Loss: 0.81689036
Epoch: 023 Step: 180 Loss: 0.83788145
Epoch: 024 Step: 000 Loss: 0.80846345
Epoch: 024 Step: 020 Loss: 0.7817747
Epoch: 024 Step: 040 Loss: 0.7549716
Epoch: 024 Step: 060 Loss: 0.76159173
Epoch: 024 Step: 080 Loss: 0.8310031
Epoch: 024 Step: 100 Loss: 0.79243195
Epoch: 024 Step: 120 Loss: 0.78161454
Epoch: 024 Step: 140 Loss: 0.8176422
Epoch: 024 Step: 160 Loss: 0.79923517
Epoch: 024 Step: 180 Loss: 0.7612796
Epoch: 025 Step: 000 Loss: 0.81551325
Epoch: 025 Step: 020 Loss: 0.8452757
Epoch: 025 Step: 040 Loss: 0.81320554
Epoch: 025 Step: 060 Loss: 0.8537542
Epoch: 025 Step: 080 Loss: 0.76204133
Epoch: 025 Step: 100 Loss: 0.7839742
Epoch: 025 Step: 120 Loss:

In [22]:
batch_board, batch_aux, batch_ys = get_training_data(a)
sess.run(net, feed_dict={board: batch_board, aux: batch_aux, Y: batch_ys})

array([[7.42465615e-01, 5.99367340e-05, 2.57474482e-01],
       [9.96362984e-01, 2.49989214e-04, 3.38701997e-03],
       [9.96493518e-01, 4.20306111e-04, 3.08620953e-03],
       [9.77828085e-01, 2.67387426e-04, 2.19045598e-02],
       [1.20456088e-02, 9.53483759e-05, 9.87859070e-01],
       [9.79022920e-01, 2.09347270e-02, 4.23674173e-05],
       [8.25399533e-03, 8.42824054e-04, 9.90903199e-01],
       [2.09046621e-02, 1.45416660e-03, 9.77641165e-01],
       [9.73291516e-01, 2.85349926e-03, 2.38549858e-02],
       [7.21077800e-01, 2.34364681e-02, 2.55485833e-01],
       [3.44967358e-02, 1.31923720e-01, 8.33579600e-01],
       [9.99983907e-01, 8.04493720e-06, 8.10684378e-06],
       [2.05605775e-01, 1.11951180e-04, 7.94282258e-01],
       [1.77569002e-01, 6.32049807e-04, 8.21798921e-01],
       [9.96743023e-01, 6.66271371e-05, 3.19040334e-03],
       [3.89866112e-03, 8.17905857e-06, 9.96093214e-01],
       [3.38504054e-02, 3.34322394e-05, 9.66116190e-01],
       [3.50823030e-02, 2.74794

In [7]:
saver.save(sess, '/Users/pschale/pythonstuff/chess_ai_project/test_saved_CNN')  

Type is unsupported, or the types of the items don't match field type in CollectionDef.
'NoneType' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'NoneType' object has no attribute 'name'


'/Users/pschale/pythonstuff/chess_ai_project/test_saved_CNN'

In [8]:
tf.saved_model.simple_save(sess, 
                            '/Users/pschale/pythonstuff/chess_ai_project/simple_saved',
                          inputs={"board":board, "aux":aux},
                          outputs={"net": net})

INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'NoneType' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'NoneType' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: b'/Users/pschale/pythonstuff/chess_ai_project/simple_saved/saved_model.pb'


In [ ]:
b = cg.game_board()
next_positions = b.find_all_next_board_positions('W')
next_positions_ar = np.array([ele.to_csv_format() for ele in next_positions])
next_positions_ar[:, :64] = le.transform(next_positions_ar[:, :64])


In [ ]:
next_positions = b.find_all_next_board_positions('W')
next_positions_ar = np.array([ele.to_csv_format() for ele in next_positions])
next_positions_ar[:, :64] = le.transform(next_positions_ar[:, :64])

next_scores = sess.run(net, feed_dict={X:next_positions_ar})
b = next_positions[np.argmax(next_scores[:,2])]

In [ ]:
print(b)

In [ ]:
b.move('Qxe5')

In [ ]:
print(b)

In [ ]:
import importlib
importlib.reload(cg)

In [ ]:
a = pd.read_csv('board_positions/board_positions_1.txt')

In [ ]:
a[a['moves_in_game'] - a['move_num']<10][a.columns[:71]]

In [ ]:
tf.__version__

In [ ]:
a = pd.read_csv('board_positions/board_positions_1.csv')
a = a[~a.isnull().any(axis=1)]
a = a[a['moves_in_game'] - a['move_num']<10]
le = preprocessing.LabelEncoder()
le.fit(['k', 'q', 'r', 'b', 'n', 'p', ' ', 'P', 'N', 'B', 'R', 'Q', 'K'])

bcols = a.columns[:64]

a[bcols] = le.transform(a[bcols])

In [ ]:
c = a.sample(100)

In [ ]:
c.shape

In [ ]:
c.as_matrix()[:, :64]

In [ ]:
start = time.time()
train = c[c.columns[:71]]
train_onehot = np.zeros((100, 71, 8))
train_onehot[:, :, 0] = train.isin([7, 8, 9, 10, 11, 12])
train_onehot[:, :, 1] = train.isin([0, 1, 2, 3, 4, 5])
train_onehot[:, :, 2] = train.isin([0, 12])
train_onehot[:, :, 3] = train.isin([1, 11])
train_onehot[:, :, 4] = train.isin([2, 10])
train_onehot[:, :, 5] = train.isin([3, 9])
train_onehot[:, :, 6] = train.isin([4, 8])
train_onehot[:, :, 7] = train.isin([5, 7])
end = time.time()
print(end - start)

In [ ]:
train_onehot[:, :, 0]